Premier modèle avec API Sequential

In [3]:
from keras.models import Sequential
from keras.layers import Dense

 Création d'un modèle Sequential

In [4]:

# Méthode 1 : Instanciation vide puis ajout de couches
model = Sequential()
# Méthode 2 : Instanciation avec liste de couches
model = Sequential([
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid') # Sortie entre 0 et 1 classification donc sigmoid
])

Exemple plus complet, construction progressive avec add

In [5]:
model = Sequential()

# Couche d'entrée implicite + première couche cachée
model.add(Dense(64, activation='relu', input_shape=(784,))) #input_shape obligatoire pour la première couche (forme des données)

# Couches cachées supplémentaires
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))

# Couche de sortie
model.add(Dense(10, activation='softmax')) # ici on a une classification à 10 classes, si binaire 1 neurone suffit

/home/sebas/formation/alyra/s6/test-install-tensorflow/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1761669061.508183  103297 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9515 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9


Anatomie d'une couche dense

In [6]:
# Paramétres critiques pour débuter

# units : Détermine la capacité d'apprentissage de la couche
# activation : 'relu' pour couches cachées, 'softmax'/'sigmoid' pour sortie
# input_shape : Uniquement pour la première couche

# Règles empiriques

# Commencer avec 32-128 unités par couche cachée
# Réduire progressivement vers la sortie 128 -> 64 -> 32

Dense(
units=64,                    
activation='relu',           
use_bias=True,              
)
# Nombre de neurones
# Fonction d'activation
# Utilisation du biais (par défaut)
kernel_initializer='glorot_uniform',  # Initialisation des poids
bias_initializer='zeros'     
# Initialisation des biais

Definition input_shape

In [7]:
# Erreurs fréquentes

#  Oublier input_shape sur la première couche
#  Inclure la batch dimension : ❌ (None, 784) ✅ (784,)
#  Mauvaise correspondance avec les données réelles

# Pour des données tabulaires (ex: 784 features -> exemple connu reco manuscrit images de 28x28)
model.add(Dense(64, activation='relu', input_shape=(784,)))
# Pour des séquences (ex: 100 timesteps, 50 features)
model.add(Dense(32, activation='relu', input_shape=(100, 50)))
# Pour des matrices (ex: images 28x28 aplaties)
model.add(Dense(128, activation='relu', input_shape=(28*28,)))


Compilation du modèle

In [8]:
# La méthode compile() - Configuration pour l'entraînement

model.compile(
    optimizer='adam',  # Algorithme d'optimisation
    loss='categorical_crossentropy',  # Fonction de perte
    metrics=['accuracy']   # Métriques à suivre        
)

Choix selon le type de problème

In [9]:
# Classification binaire
model.compile(  optimizer='adam', 
                loss='binary_crossentropy', 
                metrics=['accuracy'])

# Classification multi-classes
model.compile(  optimizer='adam', 
                loss='categorical_crossentropy', 
                metrics=['accuracy'])

# Régression
model.compile(  optimizer='adam', 
                loss='mean_squared_error', 
                metrics=['mae'])

Inspection et validation du modèle

In [11]:
# Résumé de l'architecture
model.summary()

# Nombre total de paramètres
print(f"Paramètres entraînables : {model.count_params()}")

# Visualisation de la forme des couches
for i, layer in enumerate(model.layers):
    print(f"Couche {i}: {layer.name} - Shape: {layer.output.shape}")
    

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │        50,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 10)             │           170 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │         4,224 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 60,026 (234.48 KB)

 Trainable params: 60,026 (234.48 KB)

 Non-trainable params: 0 (0.00 B)

Paramètres entraînables : 60026
Couche 0: dense_6 - Shape: (None, 64)
Couche 1: dense_7 - Shape: (None, 32)
Couche 2: dense_8 - Shape: (None, 16)
Couche 3: dense_9 - Shape: (None, 10)
Couche 4: dense_11 - Shape: (None, 64)
Couche 5: dense_12 - Shape: (None, 32)
Couche 6: dense_13 - Shape: (None, 128)


Exemple complet et bonnes pratiques (MNIST reconnaissance de char manuscrit)

In [ ]:
#  Bonnes pratiques

#  Nommer explicitement le modèle : Sequential(name="mnist_classifier"), on peut également nommer chaque couches
#  Vérifier la cohérence input_shape ↔ données
#  Tester la compilation avant l'entraînement

# Construction du modèle
model = Sequential([
    Dense(128, activation='relu', input_shape=(784,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(10, activation='softmax')  # 10 classes MNIST
],name="mnist_classifier")

# Compilation
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Vérification
model.summary()


/home/sebas/formation/alyra/s6/test-install-tensorflow/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "mnist_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,146 (434.16 KB)

 Trainable params: 111,146 (434.16 KB)

 Non-trainable params: 0 (0.00 B)